## A/B Testing

A/B testing adalah metode yang digunakan untuk membandingkan dua jenis atau lebih dari sebuah tampilan aplikasi, baik itu web, android, maupun aplikasi lainnya sehingga dapat digunakan untuk menentukan jenis mana yang lebih unggul.

Project ini akan membantu perusahaan e-commerce dalam memahami apakah perusahaan perlu mengimplementasikan halaman baru, mempertahankan halaman lama, atau menjalankan eksperimen lebih lama untuk menentukan suatu keputusan. </br>
A/B testing dilakukan sebagai salah satu upaya untuk meningkatkan kualitas halaman website dengan harapan dapat secara signifikan meningkatkan standar dari website secara otomatis. </br>

**Daftar Isi :**
1. Preprocessing data
2. A/B Testing
3. Kesimpulan

### 1. Preprocessing Data

Langkah pertama yang harus dilakukan adalah menginstall library yang akan digunakan untuk analisis :

In [10]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from scipy import stats
from scipy.stats import norm
from statsmodels.stats import weightstats as stests

**Input Dataset**

Data yang digunakan pada project ini adalah data mengenai subjek yang diberikan treatment berupa halaman baru dari suatu website. Dengan control berupa halaman lama dari suatu website. </br>
Didefinisikan sebagai berikut :
* Seseorang yang diberi **treatment** dengan diberi halaman baru dan memutuskan untuk **mendaftarkan** akun mereka ke premium maka akan diberi kode 1 
* Seseorang yang diberi **treatment** dengan diberi halaman baru dan memutuskan untuk **tidak mendaftarkan** akun mereka ke premium maka akan diberi kode 0
* Seseorang yang menjadi **kontrol** dengan menggunakan halaman lama dan memutuskan untuk **mendaftarkan** akun mereka ke premium maka akan diberi kode 1
* Seseorang yang menjadi **kontrol** dengan menggunakan halaman lama dan memutuskan untuk **tidak mendaftarkan** akun mereka ke premium maka akan diberi kode 0

In [11]:
data=pd.read_csv('ab_data.csv')
data.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


**Pengecekan missing value**

In [12]:
data.isnull().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

tidak ditemukan missing value antar variabel pengamatan

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
user_id         294478 non-null int64
timestamp       294478 non-null object
group           294478 non-null object
landing_page    294478 non-null object
converted       294478 non-null int64
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [14]:
data.user_id.nunique()

290584

**total data yang digunakan pada A/B Testing ini sebanyak 294.478, sedangkan hanya ada 290.584 data yang unik**

Hal ini menunjukkan adanya ketidaklarasan baris antara :
- variabel treatment dengan halaman page
- variabel kontrol dengan halaman page

In [15]:
data_cek = data.query('(group == "treatment" and landing_page != "new_page" ) \
         or (group != "treatment" and landing_page == "new_page")').count()[0]
print("Frekuensi treatment dengan halaman baru tidak sebaris adalah {} data".format(data_cek))

Frekuensi treatment dengan halaman baru tidak sebaris adalah 3893 data


**Pada baris yang tidak selaras, tidak dapat dipastikan apakah baris ini akan menerima halaman baru atau tidak sehingga akan dilakukan penghapusan variabel dan membuat data frame baru.**

In [16]:
data1 =data.drop(data.query('(group == "treatment" and landing_page != "new_page" ) \
                      or (group != "treatment" and landing_page == "new_page") or (group == "control" and landing_page != "old_page") or (group != "control" and landing_page == "old_page")').index)

In [17]:
data1.user_id.nunique()

290584

Jumlah data yang unik telah bersih dengan user sebanyak 290.584. Kemudian dilakukan pengecekan apakah terdapat data yang duplikat atau tidak

In [18]:
data1.user_id.duplicated().sum()

1

terdapat duplikasi data sehingga dilakukan identifikasi untuk selanjutnya dilakukan penghapusan data duplikat :

In [19]:
data1[data1['user_id'].duplicated()]['user_id']

2893    773192
Name: user_id, dtype: int64

In [20]:
data1.query('user_id == 773192')

,user_id,timestamp,group,landing_page,converted
1899,773192,2017-01-09 05:37:58.781806,treatment,new_page,0
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


In [21]:
data2 = data1.drop(data1.query('user_id == 773192 and timestamp == "2017-01-09 05:37:58.781806"').index)
data2.user_id.duplicated().sum()

0

Setelah dilakukan penghapusan data yang rangkap, dilakukan pengecekan kembali untuk memastikan apakah ada data duplikat atau tidak. Hasil menunjukkan tidak ada data yang rangkap sehingga dapat dilanjutkan untuk analisis.

**PROBABILITAS**

In [23]:
# probabilitas seseorang mengubah akun menjadi akun premium
data1_prob =data1.query('converted == 1').user_id.nunique()/data1.user_id.nunique()
print("Probabilitas seseorang mengubah akun menjadi akun premium adalah {0:.2%}".format(data1_prob))

Probabilitas seseorang mengubah akun menjadi akun premium adalah 11.96%


In [26]:
# probabilitas seseorang mengubah akun menjadi akun premium saat melihat tampilan halaman lama
data1_con = data1.query('converted == 1 and group == "control"').user_id.nunique() \
/data1.query('group == "control"').user_id.nunique()

print("Probabilitas seseorang mengubah akun menjadi akun premium saat melihat tampilan halaman lama adalah {0:.2%}".format(data1_con))

Probabilitas seseorang mengubah akun menjadi akun premium saat melihat tampilan halaman lama adalah 12.04%


In [28]:
# probabilitas seseorang mengubah akun menjadi akun premium saat melihat tampilan halaman baru
data1_treat = data1.query('converted == 1 and group == "treatment"').user_id.nunique() \
/data1.query('group == "treatment"').user_id.nunique()

print("Probabilitas seseorang mengubah akun menjadi akun premium saat melihat tampilan halaman baru {0:.2%}".format(data1_treat))

Probabilitas seseorang mengubah akun menjadi akun premium saat melihat tampilan halaman baru 11.88%


### 2. A/B TESTING

Diasumsikan halaman lama lebih baik dibandingkan dengan halaman baru menggunakan tingkat keyakinan 5% dengan kesalahan tipe 1.

**Hipotesis :**
* $H_{0}$: $p_{baru}$-$p_{lama}$ $<=$ $0$
* $H_{1}$: $p_{lama}$ - $p_{baru}$ $>$ $0$

**Statistik Uji :**

dengan menggunakan **Z.Test** untuk mengetahui apakah ada perbedaan signifikan antara menggunakan halaman baru dengan halaman lama.

In [46]:
convert_lama = data1.query('converted == 1 and landing_page == "old_page"').count()[0]
convert_baru = data1.query('converted == 1 and landing_page == "new_page"').count()[0]
n_lama = data1.query('landing_page == "old_page" ').count()[0]
n_baru= data1.query('landing_page == "new_page" ').count()[0]
convert_lama,convert_baru,n_lama,n_baru

(17489, 17264, 145274, 145311)

In [68]:
p_new = data1.converted.mean()
new_page_converted = np.random.choice([0,1],n_baru, p=(p_new,1-p_new))
new_page_converted

array([1, 1, 1, ..., 1, 1, 1])

In [64]:
def Z_test(convert_baru,convert_lama):
    z_score, p_value = sm.stats.proportions_ztest(np.array([convert_baru,convert_lama]),\
                                              np.array([n_baru,n_lama]), alternative = 'larger')
    ztabel=norm.ppf(1-(0.05/2))
    Keputusan1='Tolak H0'
    Keputusan2='Gagal Tolak H0'
    Keputusan= Keputusan1 if p_value<0.05 else Keputusan2
    
    tabel=pd.DataFrame()
    tabel['Z_Score']=[z_score,'']
    tabel['P-Value']=[p_value,'']
    tabel['Z-Tabel']=[ztabel,'']
    tabel['Keputusan']=[Keputusan,'']
    return tabel


In [65]:
Z_test(convert_baru,convert_lama)

,Z_Score,P-Value,Z-Tabel,Keputusan
0,-1.31161,0.905174,1.95996,Gagal Tolak H0
1,,,,


### Kesimpulan 

Berdasarkan hasil analisis menggunakan A/B Testing diperoleh hasil p-value sebesar 0.905 sehingga diputuskan *gagal tolak H0*, dari hasil tersebut diketahui bahwa dengan **mempertimbangkan halaman lama lebih baik dibandingkan menggunakan halaman baru**